In [1]:
import sys
sys.path.append('..')
from dataset.random_circuit import su4_circuit, random_circuit
import numpy as np

In [2]:
qubits_list = [5, 10, 15, 20, 25, 30, 35, 50, 100, 150, 200, 250, 300]

In [3]:
circuit_name = []
circuits = []
for qubits in qubits_list:
    circuit_name.append(f'rb_{qubits}_{10}')
    circuits.append(random_circuit(qubits, qubits * 10, two_qubit_prob=0.35, reverse=True))
    circuit_name.append(f'su4_{qubits}_{10}')
    circuits.append(su4_circuit(qubits, 10))

In [4]:
# 编译转换为复数矩阵
from common.circuit_preprocessing import circuit_preprocessing_matrix
from coupling_map.coupling_map import one_dim_couping
tqcs = []
for idx, circuit in enumerate(circuits):
    print(circuit_name[idx])
    tqcs.append(circuit_preprocessing_matrix(circuit=circuit, coupling_map=one_dim_couping(circuits[0].num_qubits), basis_gates=['rx', 'ry', 'cz', 'i'], rz_opti=False, xy=True)[0])
circuits = tqcs

pyzx is not installed
cirq is not installed
qsimcirq is not installed
numpy-quaternion is not installed
stim is not installed
qutip is not installed
rb_5_10
su4_5_10
rb_10_10
su4_10_10
rb_15_10
su4_15_10
rb_20_10
su4_20_10
rb_25_10
su4_25_10
rb_30_10
su4_30_10
rb_35_10
su4_35_10
rb_50_10
su4_50_10
rb_100_10
su4_100_10
rb_150_10
su4_150_10
rb_200_10
su4_200_10
rb_250_10
su4_250_10
rb_300_10
su4_300_10


In [6]:
# 接下来就是计算匹配时间
# 生成kernel对应的pulse
from common.All_pulse_generation import wave_construction
wx = np.pad(wave_construction('x').real, (0, 140), 'constant', constant_values=(0, 0))
wcz = np.pad(wave_construction('cz').real, (0, 80), 'constant', constant_values=(0, 0))
def gen_pulse_for_kernel(kernel):
    kernel_pulse = np.zeros((kernel.shape[0], kernel.shape[1] * 200), dtype=float)
    for col in range(kernel.shape[1]):
        for row in range(kernel.shape[0]):
            e = kernel[row, col]
            cs = 200 * col
            cd = cs + 200
            if e == 10:
                kernel_pulse[row, cs: cd] = wcz
            elif e != 10 and e != 0:
                kernel_pulse[row, cs: cd] = wx
    return kernel_pulse

# 准备那个kernel
import pickle
with open(f'../result/search/search_2_11_6_6_threshold8_cz', mode='rb') as f:
    all_components = pickle.load(f)
print(all_components.keys())
print(all_components[(3, 3)][0])
for kernel_size in all_components:
    for component in all_components[kernel_size]:
        array = component[0]
        array[np.where(array == 10 + 0j)] = -10-10j
        component[2] = np.sum(array)
        component.append(gen_pulse_for_kernel(component[0]))
print(all_components[(3, 3)][0])


# pluse_library
with open('single_gate_weights.result', mode='rb') as f:
    single_library = pickle.load(f)
single_pulse_library = {k: np.pad(wave_construction('x').real, (0, 140), 'constant', constant_values=(0, 0)) for k in single_library}
single_pulse_library[-10-10j] = np.pad(wave_construction('cz').real, (0, 80), 'constant', constant_values=(0, 0))

dict_keys([(3, 3), (4, 2), (2, 4), (3, 2), (2, 3), (4, 1), (1, 4), (2, 2), (3, 1), (1, 3), (2, 1), (1, 2)])
[array([[10.+0.j ,  0.+0.j ,  0.+0.j ],
       [10.+0.j ,  1.+0.5j, 10.+0.j ],
       [ 0.+0.j ,  0.+0.j , 10.+0.j ]]), 52, (41+0.5j)]
[array([[-10.-10.j ,   0. +0.j ,   0. +0.j ],
       [-10.-10.j ,   1. +0.5j, -10.-10.j ],
       [  0. +0.j ,   0. +0.j , -10.-10.j ]]), 52, (-39-39.5j), array([[ 0.02410871,  0.00936945, -0.01635341, ...,  0.        ,
         0.        ,  0.        ],
       [ 0.02410871,  0.00936945, -0.01635341, ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ]])]


In [7]:
# 匹配咯
from consts import *
from scipy import signal
import time
match_time = []

for m in circuits:
    t = time.time()
    # 这里是match的结果
    match_res = {}
    match_array = np.zeros(m.shape)
    for kernel in KERNEL_CONV_LIST:
        if kernel[0] > m.shape[0] or kernel[1] > m.shape[1]:
            continue
        # 卷积占坑
        conv_matrix = signal.convolve2d(m, FILTERS[kernel], 'valid')
        for arr in all_components[kernel]:
            find_idx = np.where(conv_matrix == arr[2])
            for row, col in zip(find_idx[0], find_idx[1]):
                if (row, col) in match_res:
                    continue
                sub_m = m[row: row + kernel[0], col: col + kernel[1]]
                match_res[(row, col)] = arr

    # print(match_res)
    # 这里该生成波形了
    pulse_array = np.zeros((m.shape[0], m.shape[1] * 200), dtype=float)
    for pos in match_res:
        arr = match_res[pos]
        q1 = pos[0]
        q2 = pos[0] + arr[0].shape[0]
        if np.sum(match_array[q1: q2, pos[1]: pos[1] + arr[0].shape[1]]) != 0:
            continue
        l = pos[1] * 200
        r = (pos[1] + arr[0].shape[1]) * 200
        # print(match_res[pos][3].shape)
        # print(f"{l} {r}")
        pulse_array[q1: q2, l: r] = match_res[pos][3]
        match_array[q1: q2, pos[1]: pos[1] + arr[0].shape[1]] = 1
    single_idx = np.where((match_array == 0) & (m != 0 + 0j))
    for idx1, idx2 in zip(single_idx[0], single_idx[1]):
        if m[idx1][idx2] == -10 - 10j:
            pulse_array[idx1, idx2 * 200: (idx2 + 1) * 200] = wcz
        elif m[idx1][idx2] in single_pulse_library:
            pulse_array[idx1, idx2 * 200: (idx2 + 1) * 200] = single_pulse_library[m[idx1][idx2]]
        else:
            pulse_array[idx1, idx2 * 200: (idx2 + 1) * 200] = np.pad(wave_construction('x').real, (0, 140), 'constant', constant_values=(0, 0))
    match_time.append(time.time() - t)
print(match_time)


[0.0034558773040771484, 0.016799211502075195, 0.005135536193847656, 0.03776407241821289, 0.006282806396484375, 0.05211520195007324, 0.00753331184387207, 0.07204723358154297, 0.007740974426269531, 0.08722925186157227, 0.01113748550415039, 0.10724806785583496, 0.013714075088500977, 0.12275409698486328, 0.019277095794677734, 0.17847490310668945, 0.03408312797546387, 0.3634462356567383, 0.04953122138977051, 0.5348591804504395, 0.07283401489257812, 0.715573787689209, 0.09077882766723633, 0.8854291439056396, 0.107452392578125, 1.068408489227295]


In [8]:
# 直接暴力生成
baseline_time = []

def one_task(idx, m):
    t = time.time()
    single_idx = np.where((m != 0 + 0j))
    pulse_array = np.zeros((m.shape[0], m.shape[1] * 200), dtype=float)
    for idx1, idx2 in zip(single_idx[0], single_idx[1]):
        if m[idx1][idx2] == -10 - 10j:
            pulse_array[idx1, idx2 * 200: (idx2 + 1) * 200] = np.pad(wave_construction('cz').real, (0, 80), 'constant', constant_values=(0, 0))
        else:
            pulse_array[idx1, idx2 * 200: (idx2 + 1) * 200] = np.pad(wave_construction('x').real, (0, 140), 'constant', constant_values=(0, 0))
    print(f'finish {idx}')
    return time.time() - t
from multiprocessing import Pool

pool = Pool(10)
futures = []
for idx, m in enumerate(circuits):
    future = pool.apply_async(one_task, (idx, m))
    futures.append(future)
for future in futures:
    baseline_time.append(future.get())

# for idx, m in enumerate(circuits):
#     print(idx)
#     t = time.time()
#     single_idx = np.where((m != 0 + 0j))
#     pulse_array = np.zeros((m.shape[0], m.shape[1] * 200), dtype=float)
#     for idx1, idx2 in zip(single_idx[0], single_idx[1]):
#         if m[idx1][idx2] == -10 - 10j:
#             pulse_array[idx1, idx2 * 200: (idx2 + 1) * 200] = np.pad(wave_construction('cz').real, (0, 80), 'constant', constant_values=(0, 0))
#         else:
#             pulse_array[idx1, idx2 * 200: (idx2 + 1) * 200] = np.pad(wave_construction('x').real, (0, 140), 'constant', constant_values=(0, 0))
#     baseline_time.append(time.time() - t)
print(baseline_time)

finish 0
finish 2
finish 1
finish 4
finish 6
finish 8
finish 10
finish 3
finish 12
finish 5
finish 14
finish 7
finish 9
finish 16
finish 11
finish 13
finish 18
finish 15
finish 20
finish 22
finish 17
finish 24
finish 19
finish 21
finish 23
finish 25
[0.20046544075012207, 0.6695842742919922, 0.46265554428100586, 1.654020071029663, 0.6841692924499512, 2.3416025638580322, 0.7322366237640381, 3.294830083847046, 0.7709732055664062, 3.969482660293579, 1.2631678581237793, 4.967066526412964, 1.3229858875274658, 5.661649227142334, 2.0995116233825684, 8.297614574432373, 3.810051441192627, 16.59216547012329, 5.632606267929077, 24.995574712753296, 8.14919924736023, 33.315253019332886, 9.928212881088257, 41.7391300201416, 13.529076337814331, 49.84784388542175]


In [9]:
# 直接暴力生成带缓存
baseline2_time = []
for idx, m in enumerate(circuits):
    t = time.time()
    single_idx = np.where((m != 0 + 0j))
    pulse_array = np.zeros((m.shape[0], m.shape[1] * 200), dtype=float)
    for idx1, idx2 in zip(single_idx[0], single_idx[1]):
        if m[idx1][idx2] == -10 - 10j:
            pulse_array[idx1, idx2 * 200: (idx2 + 1) * 200] = wcz
        elif m[idx1][idx2] in single_pulse_library:
            pulse_array[idx1, idx2 * 200: (idx2 + 1) * 200] = wx
        else:
            pulse_array[idx1, idx2 * 200: (idx2 + 1) * 200] = np.pad(wave_construction('x').real, (0, 140), 'constant', constant_values=(0, 0))
    baseline2_time.append(time.time() - t)
print(baseline2_time)

[0.0026836395263671875, 0.014177799224853516, 0.001210927963256836, 0.033312320709228516, 0.0007085800170898438, 0.04617190361022949, 0.0015058517456054688, 0.06541228294372559, 0.0010178089141845703, 0.07970380783081055, 0.0009212493896484375, 0.09824419021606445, 0.002716064453125, 0.11142182350158691, 0.0020329952239990234, 0.16575002670288086, 0.007123708724975586, 0.32740283012390137, 0.004483222961425781, 0.49855542182922363, 0.006325244903564453, 0.6578662395477295, 0.012401342391967773, 0.8190953731536865, 0.012018442153930664, 0.9949114322662354]


In [10]:
result = [circuit_name, baseline_time, baseline2_time, match_time]
import pandas as pd

df = pd.DataFrame(result)
df.to_excel('result.xlsx', header=None)